In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from urllib import request
import re
import requests
from datetime import datetime
import locale
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import time

# Get DF

In [208]:
url = "https://www.billetreduc.com/R/4/liste.htm"
response = request.urlopen(url).read()

soup = BeautifulSoup(response, 'html')

In [209]:
data = []
page_number = soup.find('td', string=re.compile(r'pages de résultats', re.IGNORECASE)).get_text()
total_pages = int(re.search(r'\d+', page_number).group())

lieu_url = []

for page_number in range(1, total_pages + 1):
    page_url = f"{url}?LISTEPEpg={page_number}"
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    preliste_table = soup.find('table', id='preliste')
    # Extract information from each row in the table
    for event in preliste_table.find_all('tr', class_=''):
        title_element = event.find('h4')
        title = title_element.text.strip() if title_element else None

        description_element = event.find('div', class_='libellepreliste')
        description = description_element.text.strip() if description_element else None

        date_element = event.find('span', class_='lieu')
        if date_element and 'Prochaines dates:' in date_element.text:
            date = date_element.text.split('Prochaines dates:')[1].strip()
        else:
            date = None

        venue_element = event.find('span', class_='lieu')
        if venue_element:
            lieu_element = venue_element.find('a').get('href')
            lieu_element ='https://www.billetreduc.com' + lieu_element

            lieu_url.append(lieu_element)
        #     try:
        #         responselieu = requests.get(lieu_element)
        #         if responselieu.status_code != 200:
        #             print(f"Failed to retrieve the webpage: {lieu_element}")
        #     except Exception as e:
        #         print(f"Error occurred while fetching the webpage: {lieu_element}, Error: {e}")
        #     soup_lieu = BeautifulSoup(responselieu.content, 'html.parser')

            
        #     if soup_lieu.find('td', {'class': 'bgbeige'}).find('h5'):
        #         venue = soup_lieu.find('td', {'class': 'bgbeige'}).find('h5').text
        #         print(venue)
        #     else:
        #         venue=None
        # else:
        #     venue = None
        # venue = venue_element.text.split(',')[0].strip() if venue_element else None 
        # venue = venue + ", " if venue_element else None
        # venue = venue+venue_element.text.split(',')[1].strip().split('Prochaines dates')[0].strip() if venue_element else None

        price_element = event.find('span', class_='prixli')
        price = price_element.text.strip() if price_element else None

        # Handle the case where 'div' with class 'reduc' is not found
        discount_element = event.find('div', class_='reduc')
        discount = discount_element.text.strip() if discount_element else None

        data.append({
            "titre": title,
            "description": description,
            "date": date,
            "lieu": lieu_element,
            "prix": price,
            "reduction": discount
        })

df = pd.DataFrame(data)

In [210]:
df = df.dropna(subset="titre").reset_index(drop=True)

In [154]:
lieu_url = list(set(lieu_url))
results_lieu = {}

for url in lieu_url:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    td_elements = soup.find_all('td', {'class': 'bgbeige'})

    for td in td_elements:
        h5_element = td.find('h5')
        if h5_element:
            # Ajouter le texte de l'élément h5 au dictionnaire
            results_lieu[url] = h5_element.text.strip()

In [ ]:
df["lieu"] = df["lieu"].map(results_lieu)
df['lieu'] = df['lieu'].str.replace(r'\s+,', ',', regex=True)

Après avoir récupéré le csv, pas besoin de lancer la cellule ci-dessous

In [219]:
df["lieu"].to_csv("lieu_concert.csv", index=False)

# Convert Date

In [184]:
def split_date(date_string):
    if pd.notna(date_string):
        if "au" in date_string:
            start_date, end_date = map(str.strip, date_string.split("au"))
        else:
            start_date, end_date = date_string, None

        # Remove "Le " or "Du " from the start of the dates
        start_date = start_date.replace("Le ", "").replace("Du ", "").strip()
    else:
        start_date, end_date = None, None

    return start_date, end_date

In [185]:
df[['start_date', 'end_date']] = df['date'].apply(split_date).apply(pd.Series)

In [161]:
locale.setlocale(locale.LC_TIME, 'fr_FR')
df["start_date"] = df["start_date"].apply(lambda x: datetime.strptime(x, "%A %d %B %Y") if x else None)
df["end_date"] = df["end_date"].apply(lambda x: datetime.strptime(x, "%A %d %B %Y")if x else None)

# Geocodage

In [223]:
dflieu = pd.read_csv("lieu_concert.geocoded.csv")
dflieu = dflieu[['lieu', 'latitude', 'longitude']]
dflieu = dflieu.drop_duplicates(subset=['lieu'])

In [229]:
df = pd.merge(df, dflieu, on='lieu', how='inner')

In [239]:
latitude_moyenne = df['latitude'].mean()
longitude_moyenne = df['longitude'].mean()
carte = folium.Map(location=[latitude_moyenne, longitude_moyenne], zoom_start=7)

for idx, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['titre'],
        icon=folium.Icon(icon="cloud"),
    ).add_to(carte)

In [240]:
carte